### Задание: YOLO и OpenCV
* Цель : 
    - Исследовать возможности детекции объектов с использованием YOLOv3 или болееновой версии в сочетании с OpenCV.
    - Углубить знания по работе с видео- и изображениями.
* Задачи:
    1. Установить YOLO и OpenCV.
    2. Выберите входные данные: можно взять набор изображений или короткое видео по вашему выбору.
    3. Детектируйте объекты на изображениях или видео с использованием YOLO.
    4. Отобразите результаты с разметкой объектов, используя OpenCV.
* Требования:
    - Вывод результатов детекции на изображение или видео.
    - Попробуйте изменить пороговое значение confidence и понаблюдайте за результатом.
* Советы:
    - Используйте YOLOv3 или более новую версию для максимального эффекта.
    - Можете экспериментировать с настройками модели для улучшения качества детекции.

In [3]:
import cv2
import numpy as np


def load_yolo():
    net = cv2.dnn.readNet('yolo/yolov3.weights', 'yolo/yolov3.cfg')
    layer_names = net.getLayerNames()
    output_layers_indices = net.getUnconnectedOutLayers()
    output_layers = [layer_names[i - 1] for i in output_layers_indices]
    return net, output_layers


def detect_objects(img, net, output_layers, confidence_threshold):
    height, width, _ = img.shape
    blob = cv2.dnn.blobFromImage(img, 0.00392, (416, 416), (0, 0, 0), True, crop=False)
    net.setInput(blob)
    outputs = net.forward(output_layers)
    
    boxes = []
    confidences = []
    class_ids = []

    for out in outputs:
        for detection in out:
            scores = detection[5:]
            class_id = np.argmax(scores)
            confidence = scores[class_id]
            
            if confidence > confidence_threshold:
                center_x = int(detection[0] * width)
                center_y = int(detection[1] * height)
                w = int(detection[2] * width)
                h = int(detection[3] * height)
                
                x = int(center_x - w / 2)
                y = int(center_y - h / 2)

                boxes.append([x, y, w, h])
                confidences.append(float(confidence))
                class_ids.append(class_id)

    indices = cv2.dnn.NMSBoxes(boxes, confidences, confidence_threshold, 0.4)
    
    for i in indices:
        box = boxes[i]
        x, y, w, h = box
        label = str(classes[class_ids[i]])
        cv2.rectangle(img, (x, y), (x + w, y + h), (0, 255, 0), 2)
        cv2.putText(img, label, (x, y + 30), cv2.FONT_HERSHEY_PLAIN, 3, (0, 255, 0), 3)

    return img

with open('yolo/coco.names', 'r') as f:
    classes = [line.strip() for line in f.readlines()]


net, output_layers = load_yolo()
img = cv2.imread('yolo/input_image.jpg')

confidence_threshold = 0.5
result_img = detect_objects(img, net, output_layers, confidence_threshold)

cv2.imshow('Image', result_img)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [4]:
cap = cv2.VideoCapture('yolo/input_video.mov')

while True:
    ret, frame = cap.read()
    if not ret:
        break

    result_frame = detect_objects(frame, net, output_layers, confidence_threshold)
    cv2.imshow('Video', result_frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.waitKey(0)
cv2.destroyAllWindows()
